In [1]:
import numpy as np
import mltools as ml
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
np.random.seed(0)


In [2]:
X = np.genfromtxt(r'D:\cs178\final_project\uci-cs178-win21/X_train.txt', delimiter=None)
Y = np.genfromtxt(r'D:\cs178\final_project\uci-cs178-win21/Y_train.txt', delimiter=None)
Xte = np.genfromtxt(r'D:\cs178\final_project\uci-cs178-win21/X_test.txt', delimiter=None)

Xtr, Xva, Ytr, Yva = ml.splitData(X, Y) # Default is 80% training/20% validation
Xtr, Ytr = ml.shuffleData(Xtr, Ytr)

#XtrP, params = ml.rescale(Xtr)
#XteP, _ = ml.rescale(Xte, params)
#XvaP, _ = ml.rescale(Xva, params)
scaler = preprocessing.StandardScaler().fit(Xtr)
Xtr = scaler.transform(Xtr)
Xva = scaler.transform(Xva)

In [4]:
nn = ml.nnet.nnetClassify()
nn.setActivation('logistic')
nn.init_weights([Xtr.shape[1], 20, len(np.unique(Ytr))], 'random', Xtr, Ytr)
nn.train(Xtr, Ytr, stopTol=1e-5, stepsize=.2, stopIter=100)

it 1 : Jsur = 0.4343540390430482, J01 = 0.3436125


D:\cs178\final_project\mltools\nnet.py:218: RuntimeWarning: overflow encountered in exp
  self.Sig = lambda z: twod(1 / (1 + np.exp(-z)))


it 2 : Jsur = 0.43437075859111474, J01 = 0.3436125
it 4 : Jsur = 0.43389466978030294, J01 = 0.3436125
it 8 : Jsur = 0.432993038283083, J01 = 0.3436125
it 16 : Jsur = 0.4319243378010827, J01 = 0.3436125
it 32 : Jsur = 0.43114126469908204, J01 = 0.3436125


In [5]:
print(f'training auc: {nn.auc(Xtr, Ytr)}')
print(f'validation auc: {nn.auc(Xva, Yva)}')

training auc: 0.6494225154264011
validation auc: 0.6525623770447198


In [11]:
#Second best
GB = GradientBoostingClassifier(n_estimators=120, learning_rate=0.32, max_depth=12)
GB.fit(Xtr, Ytr)

GB_pred = GB.predict_proba(Xva)[:, 1]
fpr, tpr, thresholds = metrics.roc_curve(Yva, GB_pred)
print(metrics.auc(fpr, tpr))


0.7821989740288865


In [5]:
'''
GB = GradientBoostingClassifier(n_estimators=120, learning_rate=0.32, max_depth=12).fit(X, Y)
GB_pred_final = GB.predict_proba(Xte)[:, 1:2]
Y_sub = np.vstack((np.arange(Xte.shape[0]), GB_pred_final[:, 0])).T

np.savetxt('Y_submit.txt', Y_sub, '%d, %.5f', header='ID,Prob1', comments='', delimiter=',')

In [12]:
#advanced gradient boosting
XB = XGBClassifier(n_estimators=135, max_depth=12, learning_rate=0.3, reg_alpha = 8).fit(Xtr, Ytr)

XB_pred = XB.predict_proba(Xva)[:, 1]
fpr, tpr, thresholds = metrics.roc_curve(Yva, XB_pred)
print(metrics.auc(fpr, tpr))

0.799249852108798


In [25]:
XB = XGBClassifier(n_estimators=135, max_depth=12, learning_rate=0.3, reg_alpha = 8).fit(X, Y)
XB_pred_final = XB.predict_proba(Xte)[:, 1:2]
Y_sub = np.vstack((np.arange(Xte.shape[0]), XB_pred_final[:, 0])).T

np.savetxt('Y_submit.txt', Y_sub, '%d, %.5f', header='ID,Prob1', comments='', delimiter=',')

In [32]:
AdaBoosting = AdaBoostClassifier(n_estimators=145, learning_rate=1.5, random_state=0, algorithm = "SAMME.R").fit(Xtr, Ytr)
Ada_pred = AdaBoosting.predict_proba(Xva)[:, 1]
fpr, tpr, thresholds = metrics.roc_curve(Yva, Ada_pred)
print(metrics.auc(fpr, tpr))

0.6921498333731135


In [28]:
RandomForest = RandomForestClassifier(n_estimators=145, max_depth= 28, criterion = "entropy", min_samples_leaf = 4, max_features = 2).fit(Xtr, Ytr)
RF_pred = RandomForest.predict_proba(Xva)[:, 1]
fpr, tpr, thresholds = metrics.roc_curve(Yva, RF_pred)
print(metrics.auc(fpr, tpr))

0.7922617350213166


In [34]:
#best choice by now
bagging = BaggingClassifier(n_estimators = 135, n_jobs = -1, max_features = 4).fit(Xtr, Ytr)
bagging_pred = bagging.predict_proba(Xva)[:, 1]
fpr, tpr, thresholds = metrics.roc_curve(Yva, bagging_pred)
print(metrics.auc(fpr, tpr))

0.8010028296701845


In [35]:
'''
bagging = BaggingClassifier(n_estimators = 135, n_jobs = -1, max_features = 4).fit(X, Y)
bagging_pred_final = bagging.predict_proba(Xte)[:, 1:2]
Y_sub = np.vstack((np.arange(Xte.shape[0]), bagging_pred_final[:, 0])).T

np.savetxt('Y_submit.txt', Y_sub, '%d, %.5f', header='ID,Prob1', comments='', delimiter=',')

In [70]:
weight = [5, 2.5, 7.4]
combined_pred = []
for i in range(len(RF_pred)):
    combined_pred.append((weight[0] * XB_pred[i] + weight[1] * RF_pred[i] + weight[2] * bagging_pred[i])/ sum(weight))
    
fpr, tpr, thresholds = metrics.roc_curve(Yva, combined_pred)
print(metrics.auc(fpr, tpr))

0.8108224648598269


In [68]:
XB = XGBClassifier(n_estimators=135, max_depth=12, learning_rate=0.3, reg_alpha = 8).fit(X, Y)
RandomForest = RandomForestClassifier(n_estimators=145, max_depth= 28, criterion = "entropy", min_samples_leaf = 4, max_features = 2).fit(X, Y)
bagging = BaggingClassifier(n_estimators = 135, n_jobs = -1, max_features = 4).fit(X, Y)

bagging_pred_final = bagging.predict_proba(Xte)[:, 1:2]
XB_pred_final = XB.predict_proba(Xte)[:, 1:2]
RF_pred_final = RandomForest.predict_proba(Xte)[:, 1:2]

In [69]:
weight = [5, 2.5, 7.4]
#weight = [7.4, 3, 10.5] second best
combined_pred = []
for i in range(len(bagging_pred_final)):
    combined_pred.append((weight[0] * XB_pred_final[i] + weight[1] * RF_pred_final[i] + weight[2] * bagging_pred_final[i])/ sum(weight))

    
Y_sub = np.vstack((np.arange(Xte.shape[0]), np.array(combined_pred)[:, 0])).T
np.savetxt('Y_submit.txt', Y_sub, '%d, %.5f', header='ID,Prob1', comments='', delimiter=',')